In [1]:
#importing neccessary libraries
import tensorflow as tf
import timeit
from datetime import datetime

## Taking Advantage of Graphs

You create and run a graph in TensorFlow by using `tf.function`, either as a direct call or as a decorator. `tf.function` takes a regular function as input and returns a __Function__. A Function is a Python callable that builds TensorFlow graphs from the Python function. You use a Function in the same way as its Python equivalent.

In [33]:
#define a python function
def python_function(x, y, b):
    x = tf.matmul(x, y)
    x = x+b
    return x

a_func_that_uses_graphs = tf.function(python_function)

2023-07-14 13:01:33.439992: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [34]:
# Make some tensors.
x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0], [3.0]])
b1 = tf.constant(4.0)

org_val = python_function(x1, y1, b1).numpy()

tf_func_val = a_func_that_uses_graphs(x1, y1, b1).numpy()

assert(org_val == tf_func_val)

A Function encapsulates several tf.Graphs behind one API 

In [44]:
#tf.function applies to a function and all other functions it calls:
def inner_function(x, y, b):
    x = tf.matmul(x,y)
    x = x+b
    return x

@tf.function
def outer_function(x):
    y = tf.constant([[2.0], [3.0]])
    b = tf.constant(4.0)
    return inner_function(x, y, b)

In [45]:
# Note that the callable will create a graph that
# includes `inner_function` as well as `outer_function`.
outer_function(tf.constant([[1.0, 2.0]])).numpy()

2023-07-14 14:11:17.624801: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


array([[12.]], dtype=float32)

## Converting Python functions to graphs

Any function written with Tensorflow can contain a mixture of python logic and tensorflow operations (tf.Operation)
While it is easier for tf.Graph to capture tensorflow operations, python logic has to undergo an extra step called 'autographing' (`tf.autograph`) that convert python code to graph generating code

In [57]:
def simple_relu(x):
    if tf.greater(x, 0):
        return x
    else:
        return 0
    
tf_simple_relu = tf.function(simple_relu)
print("First branch, with graph:", tf_simple_relu(tf.constant(10)).numpy())
print("Second branch, with graph:", tf_simple_relu(tf.constant(-1)).numpy())

First branch, with graph: 10
Second branch, with graph: 0


2023-07-14 14:22:23.567710: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [58]:
# This is the graph-generating output of AutoGraph.
print(tf.autograph.to_code(simple_relu))

def tf__simple_relu(x):
    with ag__.FunctionScope('simple_relu', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (do_return, retval_)

        def set_state(vars_):
            nonlocal do_return, retval_
            (do_return, retval_) = vars_

        def if_body():
            nonlocal do_return, retval_
            try:
                do_return = True
                retval_ = ag__.ld(x)
            except:
                do_return = False
                raise

        def else_body():
            nonlocal do_return, retval_
            try:
                do_return = True
                retval_ = 0
            except:
                do_return = False
                raise
        ag__.if_stmt(ag__.converted_call(ag__.ld(tf).greater, (ag__.ld(x), 0), None, fscope), if_bo

In [59]:
# This is the graph itself.
print(tf_simple_relu.get_concrete_function(tf.constant(1)).graph.as_graph_def())

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "shape"
    value {
      shape {
      }
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
}
node {
  name: "Greater/y"
  op: "Const"
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 0
      }
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "Greater"
  op: "Greater"
  input: "x"
  input: "Greater/y"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "cond"
  op: "StatelessIf"
  input: "Greater"
  input: "x"
  attr {
    key: "then_branch"
    value {
      func {
        name: "cond_true_211"
      }
    }
  }
  attr {
    key: "output_shapes"
    value {
      list {
        shape {
        }
        shape {
        }
      }
    }
  }
  attr {
    key: "else_branch"
   

## Graph execution vs. eager execution

The code in a Function can be executed both eagerly and as a graph. By default, Function executes its code as a graph

In [61]:
@tf.function
def get_MSE(y, y_pred):
    sq_diff = tf.pow(y-y_pred, 2)
    return tf.reduce_mean(sq_diff)

In [62]:
y_true = tf.random.uniform([5], maxval=10, dtype=tf.int32)
y_pred = tf.random.uniform([5], maxval=10, dtype=tf.int32)
print(y_true)
print(y_pred)

tf.Tensor([7 7 3 4 0], shape=(5,), dtype=int32)
tf.Tensor([6 9 8 9 9], shape=(5,), dtype=int32)


In [63]:
get_MSE(y_true, y_pred)

2023-07-14 14:29:23.440085: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


<tf.Tensor: shape=(), dtype=int32, numpy=27>

`tf.config.run_functions_eagerly(True)` This is a switch that turns off Function's ability to create and run graphs, instead of executing the code normally.

In [66]:
tf.config.run_functions_eagerly(True)

print(get_MSE(y_true, y_pred).numpy())

tf.config.run_functions_eagerly(False)

27


However, Function can behave differently under graph and eager execution. The Python print function is one example of how these two modes differ. Let's check out what happens when you insert a print statement to your function and call it repeatedly.

In [68]:
@tf.function
def get_MSE(y_true, y_pred):
    print("Calculating MSE!")
    sq_diff = tf.pow(y_true - y_pred, 2)
    return tf.reduce_mean(sq_diff)

In [69]:
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)

Calculating MSE!


2023-07-14 14:32:20.974852: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [70]:
print(error)

tf.Tensor(27, shape=(), dtype=int32)


`get_MSE` only printed once even though it was called three times.

To explain, the print statement is executed when Function runs the original code in order to create the graph in a process known as __"tracing"__ .

**Tracing** captures the TensorFlow operations into a graph, and print is not captured in the graph. That graph is then executed for all three calls without ever running the Python code again.

In [71]:
#turning off graph execution
tf.config.run_functions_eagerly(True)

error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)


tf.config.run_functions_eagerly(False)

Calculating MSE!
Calculating MSE!
Calculating MSE!


using `tf.print()` does the printing both the modes (eager as well as graph)

## tf.function best practices

Designing for tf.function may be your best bet for writing graph-compatible TensorFlow programs. Here are some tips:

- Toggle between eager and graph execution early and often with tf.config.run_functions_eagerly to pinpoint if/ when the two modes diverge.
- Create tf.Variables outside the Python function and modify them on the inside. The same goes for objects that use tf.Variable, like tf.keras.layers, tf.keras.Models and tf.keras.optimizers.
- Avoid writing functions that depend on outer Python variables, excluding tf.Variables and Keras objects. Learn more in Depending on Python global and free variables of the tf.function guide.
- Prefer to write functions which take tensors and other TensorFlow types as input. You can pass in other object types but be careful! Learn more in Depending on Python objects of the tf.function guide.
- Include as much computation as possible under a tf.function to maximize the performance gain. For example, decorate a whole training step or the entire training loop.

##  Seeing the speed-up

In [78]:
x = tf.random.uniform(shape=[10, 10], minval=-1, maxval=2, dtype=tf.dtypes.int32)

def power(x, y):
    result = tf.eye(10, dtype=tf.dtypes.int32)
    for _ in range(y):
        result = tf.matmul(x, result)
    return result

In [79]:
print("Eager execution:", timeit.timeit(lambda: power(x, 100), number=1000), "seconds")

Eager execution: 1.8123693750239909 seconds


In [80]:
power_as_graph = tf.function(power)
print("Graph execution:", timeit.timeit(lambda: power_as_graph(x, 100), number=1000), "seconds")

2023-07-14 14:48:29.187510: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Graph execution: 0.4044593330472708 seconds
